# MNIST Sigle Layer CNN training Example

Import modules.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

Prepare the MNIST dataset.

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Define a convolutional filter and pooling layer.

In [3]:
num_filters = 16

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv = tf.Variable(tf.truncated_normal([5,5,1,num_filters],
                                         stddev=0.1))
h_conv = tf.nn.conv2d(x_image, W_conv,
                      strides=[1,1,1,1], padding='SAME')
h_pool = tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                        strides=[1,2,2,1], padding='SAME')

Define a fully-connected layer and a softmax function.

In [4]:
h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters])

num_units1 = 14*14*num_filters
num_units2 = 1024

w1 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b1 = tf.Variable(tf.zeros([num_units2]))
hidden1 = tf.nn.relu(tf.matmul(h_pool_flat, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

Define a loss funtion, training algorithm and accuracy.

In [5]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Create a session and initialize Variables. Note that parameters added to the collection will be accessible after resoring the model.

In [6]:
tf.add_to_collection('input_image', x)
tf.add_to_collection('prediction', p)
tf.add_to_collection('filters', h_conv)

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

Iterate the optimization using the stochastic gradient decent for 4,000 times. It finally achieves 98% accuracy for the test set.

In [7]:
i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

Step: 100, Loss: 2550.094238, Accuracy: 0.923100
Step: 200, Loss: 1897.238281, Accuracy: 0.944600
Step: 300, Loss: 1486.104614, Accuracy: 0.957800
Step: 400, Loss: 1406.608643, Accuracy: 0.956400
Step: 500, Loss: 1234.350220, Accuracy: 0.959500
Step: 600, Loss: 972.784058, Accuracy: 0.970500
Step: 700, Loss: 1022.735962, Accuracy: 0.967900
Step: 800, Loss: 877.653687, Accuracy: 0.972600
Step: 900, Loss: 939.265625, Accuracy: 0.969500
Step: 1000, Loss: 789.847290, Accuracy: 0.976300
Step: 1100, Loss: 861.150818, Accuracy: 0.973600
Step: 1200, Loss: 712.963989, Accuracy: 0.978800
Step: 1300, Loss: 707.614136, Accuracy: 0.977900
Step: 1400, Loss: 704.807739, Accuracy: 0.977300
Step: 1500, Loss: 732.984741, Accuracy: 0.977000
Step: 1600, Loss: 802.560303, Accuracy: 0.974400
Step: 1700, Loss: 649.523315, Accuracy: 0.979000
Step: 1800, Loss: 714.464233, Accuracy: 0.977900
Step: 1900, Loss: 698.512085, Accuracy: 0.979600
Step: 2000, Loss: 663.291382, Accuracy: 0.980700
Step: 2100, Loss: 676.3

Export the trained model and check the saved files.

In [8]:
saver.save(sess, 'single_cnn_export')

'single_cnn_export'

In [9]:
!ls -l single_cnn_export*

-rw-r--r-- 1 root root 38676961 Dec 20 02:48 single_cnn_export
-rw-r--r-- 1 root root    43632 Dec 20 02:48 single_cnn_export.meta
